In [1]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os

In [3]:
#Define path
SECRETS_FILE = '/Users/gmccurdy/repos/curriculum/scripts/utilities/secrets.json'

# Load secrets
with open(SECRETS_FILE, 'r') as file:
    secrets = json.load(file)

# Assign variables from the secrets
API_URL = secrets.get('API_URL')
API_KEY = secrets.get('API_KEY')
COURSE_ID = secrets.get('COURSE_ID_STATISTICS')

#Define headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [4]:
# Create the quiz
quiz_title = input('Quiz Title')
quiz_description = input('Quiz Description')
quiz_data = {
    'quiz': {
        'title': quiz_title,
        'description': f'<p>{quiz_description}</p>',
        'quiz_type': 'assignment',
        'published': False
    }
}

create_quiz_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes'
response = requests.post(create_quiz_url, headers=headers, json=quiz_data)

if response.status_code == 200:
    quiz = response.json()
    quiz_id = quiz['id']
    print(f'Quiz created successfully with ID: {quiz_id}')
else:
    print('Failed to create quiz')
    print(f"Status Code: {response.status_code}")
    print(response.text)

Quiz created successfully with ID: 44034


In [5]:
multiple_choice_questions = [
    {
        'question_name': 'Probability Below a Given Value in Normal Distribution',
        'question_type': 'multiple_choice_question',
        'question_text': '''In a normal distribution with a mean of 50 and a standard deviation of 5, what is the probability that a randomly selected value is less than 45?''',
        'points_possible': 1,
        'answers': [
            {'text': '0.1587', 'weight': 100},  # Correct answer: z = (45-50)/5 = -1, P(z < -1) ≈ 0.1587
            {'text': '0.8413', 'weight': 0},
            {'text': '0.5000', 'weight': 0},
            {'text': '0.0228', 'weight': 0}
        ]
    },
    {
        'question_name': 'Z-Score for a Given Value',
        'question_type': 'multiple_choice_question',
        'question_text': '''What is the z-score corresponding to a value of 75 in a normal distribution with a mean of 70 and a standard deviation of 5?''',
        'points_possible': 1,
        'answers': [
            {'text': '1.00', 'weight': 100},  # Correct answer: z = (75-70)/5 = 1.00
            {'text': '0.50', 'weight': 0},
            {'text': '1.50', 'weight': 0},
            {'text': '2.00', 'weight': 0}
        ]
    },
    {
        'question_name': 'Probability Between Two Z-Scores',
        'question_type': 'multiple_choice_question',
        'question_text': '''In a standard normal distribution, what is the probability that a z-score is between -1 and 1?''',
        'points_possible': 1,
        'answers': [
            {'text': '0.6826', 'weight': 100},  # Correct answer: Approximately 68.26%
            {'text': '0.9544', 'weight': 0},
            {'text': '0.3174', 'weight': 0},
            {'text': '0.8413', 'weight': 0}
        ]
    }
]

In [6]:
essay_questions = [
    
]

In [7]:
numerical_questions = [
    {
        'question_name': 'Z-Score Calculation for Given Value',
        'question_type': 'numerical_question',
        'question_text': '''In a normal distribution with a mean of 100 and a standard deviation of 15, what is the **z-score** for a value of 130? Round your answer to two decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '2.00'  # Correct answer: z = (130 - 100) / 15 = 30/15 = 2.00
            }
        ]
    },
    {
        'question_name': 'Probability Greater Than a Z-Score',
        'question_type': 'numerical_question',
        'question_text': '''In a standard normal distribution, what is the **probability** that a z-score is greater than 1.5? Round your answer to four decimal places.''',
        'points_possible': 1,
        'answers': [
            {
                'numerical_answer_type': 'exact_answer',
                'answer_exact': '0.0668'  # From z-table, P(z > 1.5) ≈ 0.0668
            }
        ]
    }
]

In [8]:
# Combine all questions, including the essay question
questions = multiple_choice_questions + numerical_questions + essay_questions

# Add questions to the quiz
for question in questions:
    create_question_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes/{quiz_id}/questions'

    # Base question data
    question_data = {
        'question': {
            'question_name': question['question_name'],
            'question_type': question['question_type'],
            'question_text': question['question_text'],
            'points_possible': question.get('points_possible', 1)
            # Do not include 'answers' here
        }
    }

    # Include 'answers' only if they exist and the question is not an essay question
    if 'answers' in question and question['question_type'] != 'essay_question':
        question_data['question']['answers'] = question['answers']

    # Include 'general_feedback' if it exists (for essay questions)
    if 'general_feedback' in question:
        question_data['question']['general_feedback'] = question['general_feedback']

    # Send the request to create the question
    response = requests.post(create_question_url, headers=headers, json=question_data)

    if response.status_code == 201 or response.status_code == 200:
        print(f"Added question '{question['question_name']}' successfully.")
    else:
        print(f"Failed to add question '{question['question_name']}'")
        print(f"Status Code: {response.status_code}")
        print(response.text)

Added question 'Probability Below a Given Value in Normal Distribution' successfully.
Added question 'Z-Score for a Given Value' successfully.
Added question 'Probability Between Two Z-Scores' successfully.
Added question 'Z-Score Calculation for Given Value' successfully.
Added question 'Probability Greater Than a Z-Score' successfully.
